# SQLAlchemy w/ ORM

An Object Relational Mapper is a pattern for mapping domain models(objects usually in the form of classes) to tables in a relational database. 

There are 2 common patterns for implementing an ORM
1. Active Record (Ruby on Rails, Django, etc.) - Here you typically pull some record out of the database (i.e. `user = User.objects.get()`), do something to that record, and then put the record back (i.e. `user.save()`). Here, the thing to note is that the persistance is tightly coupled to the domain model which lends itself to a very convenient API.

2. Data Mapper (SQLAlchemy and others) - Here you have some separate that that _maps_ a domain model over to a table and that separate thing manages the persistance which allows you to decouple the persistance from the domain models themselves. SQLAlchemy follows this with the unit-of-work pattern whereby instead of the common pattern mentioned in (1) where we pull out a record, do something and then put it back, SQLAlchemy is much more concerned about managing _units of work_ rather than managing individual records so you roll together some amount of actions into a unit of work and then commit them all as one unit.

For more see: 

https://tekshinobi.com/sql-alchemy-tutorial/#:~:text=Level%204%3A%20Object%20Relational%20Mapping

https://docs.sqlalchemy.org/en/14/orm/quickstart.html

In [3]:
from sa2_swing.config import SQLALCHEMY_URI

from sqlalchemy import create_engine

engine = create_engine(SQLALCHEMY_URI, echo=True)

## Create some tables

In [4]:
# CREATE TABLES WITH CORE
from sqlalchemy import Column, Integer, String, Table
from sqlalchemy.orm import declarative_base 

Base = declarative_base()

class User(Base):
    __tablename__ = "user"

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String(30))
    last_name= Column(String(100))

    def __repr__(self):
        return f"User({self.first_name=} {self.last_name=}"

Base.metadata.create_all(engine)


2022-04-22 17:44:59,691 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-04-22 17:44:59,692 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-22 17:44:59,695 INFO sqlalchemy.engine.Engine select current_schema()
2022-04-22 17:44:59,696 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-22 17:44:59,700 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-04-22 17:44:59,701 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-22 17:44:59,705 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-22 17:44:59,707 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-22 17:44:59,709 INFO sqlalchemy.engine.Engine [generated in 0.00129s] {'name': 'user'}
2022-04-22 17:44:59,713 INFO sqlalchemy.engine.Engine 
CREATE TABLE "user" (
	id SERIAL NOT NULL, 
	first_name VARCHAR(30), 
	last_name VARCHAR(100), 
	PRIMARY KEY (id)
)


2022-04-22 17:44:59

## Insert some data

In [5]:
from sqlalchemy import text
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(engine)

with Session() as session:
	session.execute(text('TRUNCATE TABLE "user"'))

	collin = User(first_name="Collin", last_name="Choy")
	session.add(collin)
	session.commit()

	session.add_all([User(first_name="Jeff", last_name="Bridges"), User(first_name="Tim", last_name="None")])
	session.commit()


2022-04-22 17:45:01,381 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-22 17:45:01,388 INFO sqlalchemy.engine.Engine TRUNCATE TABLE "user"
2022-04-22 17:45:01,393 INFO sqlalchemy.engine.Engine [generated in 0.00422s] {}
2022-04-22 17:45:01,401 INFO sqlalchemy.engine.Engine INSERT INTO "user" (first_name, last_name) VALUES (%(first_name)s, %(last_name)s) RETURNING "user".id
2022-04-22 17:45:01,403 INFO sqlalchemy.engine.Engine [generated in 0.00148s] {'first_name': 'Collin', 'last_name': 'Choy'}
2022-04-22 17:45:01,405 INFO sqlalchemy.engine.Engine COMMIT
2022-04-22 17:45:01,410 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-22 17:45:01,413 INFO sqlalchemy.engine.Engine INSERT INTO "user" (first_name, last_name) VALUES (%(first_name)s, %(last_name)s) RETURNING "user".id
2022-04-22 17:45:01,414 INFO sqlalchemy.engine.Engine [generated in 0.00121s] ({'first_name': 'Jeff', 'last_name': 'Bridges'}, {'first_name': 'Tim', 'last_name': 'None'})
2022-04-22 17:45:01,418 INFO sqla

In [6]:
session.new

IdentitySet([])

## Query some data

In [7]:
from sqlalchemy import select

def select_all_users():
	with Session() as session:
		result = session.execute(select(User)).all()
	return result

select_all_users()


def select_all_users():
	with Session() as session:
		result = session.execute(select(User)).scalars().all()
	return result

select_all_users()

2022-04-22 17:45:03,489 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-22 17:45:03,498 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".first_name, "user".last_name 
FROM "user"
2022-04-22 17:45:03,503 INFO sqlalchemy.engine.Engine [generated in 0.00464s] {}
2022-04-22 17:45:03,506 INFO sqlalchemy.engine.Engine ROLLBACK
2022-04-22 17:45:03,508 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-22 17:45:03,510 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".first_name, "user".last_name 
FROM "user"
2022-04-22 17:45:03,511 INFO sqlalchemy.engine.Engine [cached since 0.01339s ago] {}
2022-04-22 17:45:03,514 INFO sqlalchemy.engine.Engine ROLLBACK


[User(self.first_name='Collin' self.last_name='Choy',
 User(self.first_name='Jeff' self.last_name='Bridges',
 User(self.first_name='Tim' self.last_name='None']

In [8]:
stmt = select(User.first_name, User.id).where((User.first_name > "A") & (User.first_name < "M"))

with Session() as session:
	result = session.execute(stmt).all()
result

2022-04-22 17:45:04,376 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-22 17:45:04,386 INFO sqlalchemy.engine.Engine SELECT "user".first_name, "user".id 
FROM "user" 
WHERE "user".first_name > %(first_name_1)s AND "user".first_name < %(first_name_2)s
2022-04-22 17:45:04,389 INFO sqlalchemy.engine.Engine [generated in 0.00307s] {'first_name_1': 'A', 'first_name_2': 'M'}
2022-04-22 17:45:04,392 INFO sqlalchemy.engine.Engine ROLLBACK


[('Collin', 1), ('Jeff', 2)]

In [9]:
with Session() as session:
	Jeff = session.execute(
		select(User).filter_by(first_name="Jeff")
	).scalar()
Jeff

2022-04-22 17:45:05,190 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-22 17:45:05,195 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".first_name, "user".last_name 
FROM "user" 
WHERE "user".first_name = %(first_name_1)s
2022-04-22 17:45:05,200 INFO sqlalchemy.engine.Engine [generated in 0.00506s] {'first_name_1': 'Jeff'}
2022-04-22 17:45:05,208 INFO sqlalchemy.engine.Engine ROLLBACK


User(self.first_name='Jeff' self.last_name='Bridges'

## Update some data

In [10]:
from sqlalchemy import update

stmt = update(User).where(User.first_name == 'Jeff').values(last_name='Goldblum')
with Session() as session:
	session.execute(stmt)
	session.commit()

select_all_users()


2022-04-22 17:45:06,473 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-22 17:45:06,484 INFO sqlalchemy.engine.Engine UPDATE "user" SET last_name=%(last_name)s WHERE "user".first_name = %(first_name_1)s
2022-04-22 17:45:06,488 INFO sqlalchemy.engine.Engine [generated in 0.00428s] {'last_name': 'Goldblum', 'first_name_1': 'Jeff'}
2022-04-22 17:45:06,490 INFO sqlalchemy.engine.Engine COMMIT
2022-04-22 17:45:06,494 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-22 17:45:06,497 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".first_name, "user".last_name 
FROM "user"
2022-04-22 17:45:06,498 INFO sqlalchemy.engine.Engine [cached since 3.001s ago] {}
2022-04-22 17:45:06,502 INFO sqlalchemy.engine.Engine ROLLBACK


[User(self.first_name='Collin' self.last_name='Choy',
 User(self.first_name='Tim' self.last_name='None',
 User(self.first_name='Jeff' self.last_name='Goldblum']

In [11]:
result[1]

('Jeff', 2)

## Delete some data

## Cleanup: Drop tables

In [13]:
Base.metadata.drop_all(engine)

2022-04-22 17:46:17,486 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-22 17:46:17,492 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-22 17:46:17,495 INFO sqlalchemy.engine.Engine [cached since 77.86s ago] {'name': 'user'}
2022-04-22 17:46:17,497 INFO sqlalchemy.engine.Engine 
DROP TABLE "user"
2022-04-22 17:46:17,499 INFO sqlalchemy.engine.Engine [no key 0.00132s] {}
2022-04-22 17:46:17,510 INFO sqlalchemy.engine.Engine COMMIT
